## Create a function to upload files from s3 storage link

In [2]:
import requests
from bs4 import BeautifulSoup
import json
import numpy as np
import pandas as pd

- I would like to fetch and extract data files from the provided html link using requesets and beautifulsoup
- This will help the flow in the future unless partitions have changed

In [2]:
html_url = "https://fetch-hiring.s3.amazonaws.com/data-analyst/ineeddata-data-modeling/data-modeling.html"

r = requests.get(html_url)
soup = BeautifulSoup(r.content, 'html.parser')

In [3]:
parsed_html = soup

In [4]:
# found the file as a list we have about 3 files
amount_of_files = len(parsed_html.find_all('a'))

In [5]:
# looking how to parse individually for file name
data_files = parsed_html.find_all('a')[0]['href'].split('/')[-1].split('.')[0]
data_files

'receipts'

In [6]:
# parsing the actual link to upload the file
file_urls = parsed_html.find_all('a')[1]['href']
file_urls

'https://fetch-hiring.s3.amazonaws.com/data-analyst/ineeddata-data-modeling/users.json.gz'

In [7]:
data_files

'receipts'

In [10]:
def parseHtml(html_url):
    
    r = requests.get(html_url)
    soup = BeautifulSoup(r.content, 'html.parser')
    parsed_html = soup
    amount_of_files = len(parsed_html.find_all('a'))
    data_names = []
    url_list = []
    for i in range(amount_of_files):
        data_files = parsed_html.find_all('a')[i]['href'].split('/')[-1].split('.')[0]
        file_urls = parsed_html.find_all('a')[i]['href']
        
        data_names.append(data_files)
        url_list.append(file_urls)
    
    return data_names, url_list

In [11]:
dataNames, urlList = parseHtml(html_url)
print('names:', dataNames)
print('urls:', urlList)

names: ['receipts', 'users', 'brands']
urls: ['https://fetch-hiring.s3.amazonaws.com/data-analyst/ineeddata-data-modeling/receipts.json.gz', 'https://fetch-hiring.s3.amazonaws.com/data-analyst/ineeddata-data-modeling/users.json.gz', 'https://fetch-hiring.s3.amazonaws.com/data-analyst/ineeddata-data-modeling/brands.json.gz']


- After scraping the HTML I found that it was coming in as bytes
- We need to download the json.gz file and open it as a json file to read from pandas
- We also want delete the file after we turn it into a json file (no need for extra files)
- Last would be to get the json read as a pandas dataframe and save it as a csv file for further cleaning.

In [12]:
import wget
import tarfile
import shutil
import gzip
import os

# Code Breakdown (Scroll Down to run function)

In [628]:
# download the json.gz file
filename = wget.download(file_urls)

# unzip the gz file and write a new file as json
with gzip.open(filename) as g:
    with open(f'{data_files}.json', 'wb') as f_out:
        shutil.copyfileobj(g, f_out)

# if the gz file exists delete (save space) - or log for upload
if os.path.exists(f"{filename}"):
    os.remove(f"{filename}")
    print("old file removed ;)")
else:
    print("The file does not exist")

# if the file exist already we will rewrite it or we can log as new file if there is new date 
if os.path.exists(f"json_files/{data_files}.json"):
    shutil.copy(f'{data_files}.json', 'json_files/')
    print('File rewritten')
else:
# else the file does not exists we will move the file to the proper folder
    shutil.move(f'{data_files}.json', 'json_files/')
    print('File moved')


df = pd.read_json(f'json_files/{data_files}.json', lines = True)

path = 'csv_files/'
if not os.path.exists(path):
    os.makedirs(path)
    
df.to_csv(f'{path}{data_files}.csv')

old file removed ;)
File rewritten


In [693]:
urlList

['https://fetch-hiring.s3.amazonaws.com/data-analyst/ineeddata-data-modeling/receipts.json.gz',
 'https://fetch-hiring.s3.amazonaws.com/data-analyst/ineeddata-data-modeling/users.json.gz',
 'https://fetch-hiring.s3.amazonaws.com/data-analyst/ineeddata-data-modeling/brands.json.gz']

In [17]:
def createFiles(data_files, file_urls):
    
    for dfnames,dfurls in zip(data_files,file_urls):
            
            # download the json.gz file
            filename = wget.download(dfurls)
            names = dfnames
            # unzip the gz file and write a new file as json
            with gzip.open(filename) as g:
                with open(f'{names}.json', 'wb') as f_out:
                    shutil.copyfileobj(g, f_out)

            # if the gz file exists delete (save space) - or log for upload
            if os.path.exists(f"{filename}"):
                os.remove(f"{filename}")
                print("old file removed ;)")
            else:
                print("The file does not exist")

            # if the file exist already we will rewrite over to json file
            #  or we can log as new file if there is new date

            json_path = 'json_files/'
            if not os.path.exists(json_path):
                os.makedirs(json_path)

            if os.path.exists(f"json_files/{names}.json"):
                shutil.copy(f'{names}.json', 'json_files/')
                print('File rewritten')
            else:
            # else the file does not exists we will move the file to the proper folder
                shutil.move(f'{names}.json', 'json_files/')
                print('File moved')


            df = pd.read_json(f'json_files/{names}.json', lines = True)

            # create folder for csv_files
            csv_path = 'csv_files/'
            if not os.path.exists(csv_path):
                os.makedirs(csv_path)

            df.to_csv(f'{csv_path}{names}.csv')

            # remove any extra files lingering
            if os.path.exists(f"{names}.json"):
                os.remove(f"{names}.json")

    print("All files are uploaded")

### Personal Notes
- Had an issue here seems like it is iterating a few times and a Typeerror io.'_io.BufferedWriter 
     - could be an update problem on my end
  - (06/10/21 - 3:40pm)
- Typeerror io.'_io.BufferedWriter issue. 
 - solved the issue should work properly 
   - (06/10/21 - 4:39pm est)

# Run Function

In [18]:
dataNames, urlList = parseHtml(html_url)
createFiles(dataNames, urlList)

old file removed ;)
File rewritten
old file removed ;)
File rewritten
old file removed ;)
File rewritten
All files are uploaded


- Check if is in the files

In [19]:
dataNames

['receipts', 'users', 'brands']

In [23]:
df_json = pd.read_json(f'json_files/{dataNames[0]}.json', lines = True)
df_csv = pd.read_csv(f'csv_files/{dataNames[0]}.csv', index_col = 0)

In [25]:
df_json.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [24]:
df_csv.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052
